# Avalanche Danger Level Forecast: Preliminary Modeling

I am going to plug out newly created model-ready dataset into a few out-of-the-box models

## Import Tools

In [44]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

# pandas
from pandas.plotting import scatter_matrix

## Import Data

In [45]:
avi = pd.read_csv('SnowWeatherClean.csv')
avi.head()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,northwest_wind_hours,prevailing_wind_E,prevailing_wind_N,prevailing_wind_NE,prevailing_wind_NW,prevailing_wind_S,prevailing_wind_SE,prevailing_wind_SW,prevailing_wind_W,next_day_avi_danger
0,0,2.0,10.74,0.0,0.0,2010.0,12.0,18.0,12,4,...,12,0,0,0,1,0,0,0,0,1.0
1,1,1.0,9.40,3.0,0.0,2010.0,12.0,19.0,15,3,...,12,0,0,0,1,0,0,0,0,1.0
2,2,1.0,20.58,0.0,0.0,2010.0,12.0,20.0,15,5,...,0,0,1,0,0,0,0,0,0,3.0
3,3,3.0,35.12,3.0,0.0,2010.0,12.0,21.0,18,10,...,0,0,1,0,0,0,0,0,0,2.0
4,4,2.0,33.78,-3.0,0.0,2010.0,12.0,22.0,15,7,...,1,0,1,0,0,0,0,0,0,3.0


## Dataset at a Glance

In [46]:
avi.describe()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,northwest_wind_hours,prevailing_wind_E,prevailing_wind_N,prevailing_wind_NE,prevailing_wind_NW,prevailing_wind_S,prevailing_wind_SE,prevailing_wind_SW,prevailing_wind_W,next_day_avi_danger
count,1253.000000,1253.000000,1253.000000,1253.00000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,...,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000,1253.000000
mean,626.000000,2.088587,43.398746,-0.03352,-0.056345,2015.587390,3.499601,16.616919,19.794094,3.831604,...,6.521149,0.014366,0.049481,0.018356,0.276137,0.066241,0.026337,0.072626,0.476457,2.087789
std,361.854252,0.888858,16.512870,11.28772,13.006894,2.960021,3.110999,8.432409,14.034897,14.988959,...,7.619898,0.119040,0.216957,0.134288,0.447264,0.248802,0.160199,0.259625,0.499645,0.889387
min,0.000000,1.000000,4.250000,-44.00000,-53.800000,2010.000000,1.000000,1.000000,-26.000000,-40.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,313.000000,1.000000,31.090000,-5.00000,-8.400000,2013.000000,2.000000,10.000000,11.000000,-7.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,626.000000,2.000000,43.400000,1.00000,0.400000,2016.000000,3.000000,17.000000,19.000000,5.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,939.000000,3.000000,55.480000,7.00000,9.200000,2018.000000,4.000000,24.000000,30.000000,13.000000,...,11.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000
max,1252.000000,5.000000,110.060000,39.00000,34.600000,2020.000000,12.000000,31.000000,57.000000,47.000000,...,24.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000


In [47]:
avi.columns

Index(['Unnamed: 0', 'avi_danger', 'avg_wind', 'temp_max_swing',
       'temp_max_swing_from_avg', 'year', 'month', 'day', 'temp_max',
       'temp_min', 'water_equivalent', 'snow_fall', 'snow_depth_6am',
       'wind_speed_sum', 'sunshine_percent', 'west_wind_hours',
       'northwest_wind_hours', 'prevailing_wind_E', 'prevailing_wind_N',
       'prevailing_wind_NE', 'prevailing_wind_NW', 'prevailing_wind_S',
       'prevailing_wind_SE', 'prevailing_wind_SW', 'prevailing_wind_W',
       'next_day_avi_danger'],
      dtype='object')

In [48]:
avi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               1253 non-null   int64  
 1   avi_danger               1253 non-null   float64
 2   avg_wind                 1253 non-null   float64
 3   temp_max_swing           1253 non-null   float64
 4   temp_max_swing_from_avg  1253 non-null   float64
 5   year                     1253 non-null   float64
 6   month                    1253 non-null   float64
 7   day                      1253 non-null   float64
 8   temp_max                 1253 non-null   int64  
 9   temp_min                 1253 non-null   int64  
 10  water_equivalent         1253 non-null   float64
 11  snow_fall                1253 non-null   float64
 12  snow_depth_6am           1253 non-null   float64
 13  wind_speed_sum           1253 non-null   int64  
 14  sunshine_percent        

## Filter Dataset

In [49]:
avi = avi.drop(['Unnamed: 0'], axis=1)

In [50]:
avi = avi[avi['avi_danger'].notnull()]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253 entries, 0 to 1252
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avi_danger               1253 non-null   float64
 1   avg_wind                 1253 non-null   float64
 2   temp_max_swing           1253 non-null   float64
 3   temp_max_swing_from_avg  1253 non-null   float64
 4   year                     1253 non-null   float64
 5   month                    1253 non-null   float64
 6   day                      1253 non-null   float64
 7   temp_max                 1253 non-null   int64  
 8   temp_min                 1253 non-null   int64  
 9   water_equivalent         1253 non-null   float64
 10  snow_fall                1253 non-null   float64
 11  snow_depth_6am           1253 non-null   float64
 12  wind_speed_sum           1253 non-null   int64  
 13  sunshine_percent         1253 non-null   int64  
 14  west_wind_hours         

## Fill Remaing NA's

In [51]:
avi = avi.fillna(0)
avi = avi[avi.avi_danger != 5]
avi = avi[avi.next_day_avi_danger != 5]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1249 entries, 0 to 1252
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   avi_danger               1249 non-null   float64
 1   avg_wind                 1249 non-null   float64
 2   temp_max_swing           1249 non-null   float64
 3   temp_max_swing_from_avg  1249 non-null   float64
 4   year                     1249 non-null   float64
 5   month                    1249 non-null   float64
 6   day                      1249 non-null   float64
 7   temp_max                 1249 non-null   int64  
 8   temp_min                 1249 non-null   int64  
 9   water_equivalent         1249 non-null   float64
 10  snow_fall                1249 non-null   float64
 11  snow_depth_6am           1249 non-null   float64
 12  wind_speed_sum           1249 non-null   int64  
 13  sunshine_percent         1249 non-null   int64  
 14  west_wind_hours         

### Standardizing the Data

In [52]:
#standardizer 
def standardize(X_train, X_test):
    scaler = StandardScaler()
    # Fitting and transforming training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    # Tranforming testing data based on traning fit (prevent data leakage)
    X_test = scaler.transform(X_test)
    return X_train, X_test


## Splitting Dataset

In [53]:
x = avi.iloc[:, 0:avi.shape[1]-2]
y = avi.iloc[:, avi.shape[1]-1]

print(x.shape)
print(y.shape)


(1249, 23)
(1249,)


In [54]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [55]:
# Scaling
x_train, x_test = standardize(x_train, x_test)

In [56]:
print(y)

0       1.0
1       1.0
2       3.0
3       2.0
4       3.0
       ... 
1248    3.0
1249    3.0
1250    3.0
1251    2.0
1252    1.0
Name: next_day_avi_danger, Length: 1249, dtype: float64


# Logistic Regression

In [57]:

penalty = ['l1', 'l2', 'elasticnet', 'none']
tol = [0.0001, 0.001, 0.01, 0.1]
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000] 
solver = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
param_distributions = dict(penalty=penalty,
                           tol=tol,
                           C=C,
                           solver = solver)


lr = LogisticRegression(class_weight = 'balanced',max_iter = 1000)
lr_cv = RandomizedSearchCV(estimator=lr, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
lr_cv.fit(x_train, y_train)
y_pred_lr = lr_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_lr))
print(cross_val_score(lr, x_train, y_train, cv=3))

0.6293929712460063
[0.53846154 0.57371795 0.5224359 ]


/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 453, in _check_solver
    raise ValueError(
ValueError: penalty='none' is not supported for the liblinear solver

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_

In [58]:
print(confusion_matrix(y_test, y_pred_lr))

[[81  7  7  0]
 [17 69 19  3]
 [12 28 43  5]
 [ 1  3 14  4]]


In [59]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

         1.0       0.73      0.85      0.79        95
         2.0       0.64      0.64      0.64       108
         3.0       0.52      0.49      0.50        88
         4.0       0.33      0.18      0.24        22

    accuracy                           0.63       313
   macro avg       0.56      0.54      0.54       313
weighted avg       0.61      0.63      0.62       313



# Support Vector Machine (SVM)

In [60]:
param_distributions = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  


svm = SVC()
svm_cv = RandomizedSearchCV(estimator=svm, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
svm_cv.fit(x_train, y_train)
y_pred_svm = svm_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_svm))
print(cross_val_score(svm, x_train, y_train, cv=3))

0.6293929712460063
[0.61858974 0.64423077 0.6025641 ]


In [61]:
print(confusion_matrix(y_test, y_pred_svm))

[[77 14  4  0]
 [13 83 12  0]
 [12 40 36  0]
 [ 2  7 12  1]]


In [62]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

         1.0       0.74      0.81      0.77        95
         2.0       0.58      0.77      0.66       108
         3.0       0.56      0.41      0.47        88
         4.0       1.00      0.05      0.09        22

    accuracy                           0.63       313
   macro avg       0.72      0.51      0.50       313
weighted avg       0.65      0.63      0.60       313



# K-Nearest Neighbors

In [63]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
param_distributions = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

knn = KNeighborsClassifier()
knn_cv = RandomizedSearchCV(estimator=knn, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
knn_cv.fit(x_train, y_train)
y_pred_knn = knn_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_knn))
print(cross_val_score(knn, x_train, y_train, cv=3))

0.5750798722044729
[0.54807692 0.58012821 0.54807692]


In [64]:
print(confusion_matrix(y_test, y_pred_knn))

[[70 21  4  0]
 [14 76 18  0]
 [11 43 34  0]
 [ 4  9  9  0]]


In [65]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

         1.0       0.71      0.74      0.72        95
         2.0       0.51      0.70      0.59       108
         3.0       0.52      0.39      0.44        88
         4.0       0.00      0.00      0.00        22

    accuracy                           0.58       313
   macro avg       0.44      0.46      0.44       313
weighted avg       0.54      0.58      0.55       313



/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Gradient Boosting Classifier

In [66]:
param_distributions = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
                       'n_estimators':[100,250,500,750,1000,1250,1500,1750],
                       'max_depth':[2,3,4,5,6,7] }

gbc = GradientBoostingClassifier()
gbc_cv = RandomizedSearchCV(estimator=gbc, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
gbc.fit(x_train, y_train)
y_pred_gbc = gbc.predict(x_test)
print(accuracy_score(y_test, y_pred_gbc))
print(cross_val_score(gbc, x_train, y_train, cv=3))

0.6038338658146964
[0.61217949 0.56730769 0.59294872]


In [67]:
print(confusion_matrix(y_test, y_pred_gbc))

[[76 15  3  1]
 [ 9 73 23  3]
 [10 38 39  1]
 [ 1  7 13  1]]


In [68]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

         1.0       0.71      0.74      0.72        95
         2.0       0.51      0.70      0.59       108
         3.0       0.52      0.39      0.44        88
         4.0       0.00      0.00      0.00        22

    accuracy                           0.58       313
   macro avg       0.44      0.46      0.44       313
weighted avg       0.54      0.58      0.55       313



/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
